In [27]:
# !pip install roboflow

In [42]:
from roboflow import Roboflow
rf = Roboflow(api_key="jJyjVbMQRppa3lA1JR0i")
project = rf.workspace().project("driver-s-license-text-extraction-6zjsk")
model = project.version(1).model

# infer on a local image
pred_data = model.predict("sample.jpg", confidence=40, overlap=30).json()

# visualize your prediction
model.predict("sample.jpg", confidence=40, overlap=30).save("prediction.jpg")

# infer on an image hosted elsewhere
# print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())

loading Roboflow workspace...
loading Roboflow project...


In [43]:
import cv2

# Read the image
image = cv2.imread('sample.jpg')

# Convert the image to grayscale
grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Save the grayscale image
cv2.imwrite('sample_grayscale.jpg', grayscale_image)

True

In [44]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

def adjust_black_and_white(image_path, threshold=50):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return None  # Exit the function if image loading fails

    # Create a mask for pixels close to black (i.e., pixels where all channels are close to 0)
    black_mask = np.all(image <= threshold, axis=-1)

    # Create a mask for pixels close to white (i.e., pixels where all channels are close to 255)
    white_mask = np.all(image >= 255 - threshold, axis=-1)

    # Set pixels close to black to pure black
    image[black_mask] = [0, 0, 0]

    # Set pixels close to white to pure white
    image[white_mask] = [255, 255, 255]

    return image


# Function to crop the image based on midpoint coordinates
def crop_image_by_midpoints(image_path, coordinates, output_dir):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over the coordinates and crop the image
    for i, coord in enumerate(coordinates):
        # Extract the midpoint coordinates (x, y), and width, height
        x = coord['x']
        y = coord['y']
        width = coord['width']
        height = coord['height']

        extra = 0
        # Calculate the top-left and bottom-right corners based on the midpoint and size
        x1 = int(x - width / 2) -extra
        y1 = int(y - height / 2)  - extra
        x2 = int(x + width / 2) + extra
        y2 = int(y + height / 2) + extra

        # Ensure that the coordinates are within the image boundaries
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(image.shape[1], x2)
        y2 = min(image.shape[0], y2)

        # Crop the image using NumPy array slicing
        cropped_image = image[y1:y2, x1:x2]
        # Save the cropped image
        output_image_path = os.path.join(output_dir, f"{coord['class']}_{i+1}.jpg")
        cv2.imwrite(output_image_path, cropped_image)
        print(f"Saved cropped image: {output_image_path}")

        image_path = output_image_path  # Make sure this path is correct!

        # Adjust pixels close to black and white
        adjusted_image = adjust_black_and_white(image_path)

        if adjusted_image is not None:
        # Save the resulting image
            output_path = output_image_path
            cv2.imwrite(output_path, adjusted_image)

# Example coordinates (replace this with your actual annotation data)

# Path to the image file
image_path = 'sample_grayscale.jpg'

# Directory to save cropped images
output_dir = 'cropped_images'

# Call the function to crop the image based on midpoint coordinates
crop_image_by_midpoints(image_path, pred_data['predictions'], output_dir)


Saved cropped image: cropped_images\Exp date_1.jpg
Saved cropped image: cropped_images\First name_2.jpg
Saved cropped image: cropped_images\Last name_3.jpg
Saved cropped image: cropped_images\Issue date_4.jpg
Saved cropped image: cropped_images\Address_5.jpg
Saved cropped image: cropped_images\DOB_6.jpg
Saved cropped image: cropped_images\Sex_7.jpg
Saved cropped image: cropped_images\Class_8.jpg
Saved cropped image: cropped_images\License number_9.jpg


In [45]:
import pytesseract
import os
from PIL import Image
import cv2
import numpy as np

# Path to Tesseract executable (if it's not in your PATH, you'll need to specify the full path)
# Example for Windows: pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to preprocess and perform OCR on each image in the cropped_images directory
def preprocess_image(image_path, alpha=1.5, beta=0):
    # Load the image using OpenCV
    img = cv2.imread(image_path)

    # Convert the image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Adjust contrast by modifying alpha and beta
    contrast_img = cv2.convertScaleAbs(gray_img, alpha=alpha, beta=beta)

    # Apply thresholding to binarize the image (you can experiment with different methods)
    _, binary_img = cv2.threshold(contrast_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Optional: Apply morphological transformations to clean the image (if needed)
    kernel = np.ones((3, 3), np.uint8)
    morph_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel)

    # Return the processed image
    return Image.fromarray(morph_img)

# Function to perform OCR on each image in the cropped_images directory
def ocr_on_cropped_images(cropped_images_dir):
    # Iterate over all files in the cropped_images directory
    for filename in os.listdir(cropped_images_dir):
        # Only process image files (you can filter by extensions like .jpg, .jpeg, .png)
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(cropped_images_dir, filename)
            
            try:
                # Preprocess the image (change contrast here)
                processed_img = preprocess_image(image_path, alpha=1.5, beta=0)
                
                # Use pytesseract to extract text
                text = pytesseract.image_to_string(processed_img)

                # Extract the class name from the image filename
                # Assuming the class is part of the filename (e.g., 'cropped_License Number_1.jpg')
                class_name = filename.split('_')[0]  # This will give the class name between 'cropped' and the index

                # Display the extracted text and class name
                print(f"Class: {class_name}")
                print(f"Extracted Text: {text.strip()}")
                print("-" * 50)

            except Exception as e:
                print(f"Error processing {filename}: {e}")
                continue  # Skip the problematic image and move to the next

# Path to the cropped_images directory
cropped_images_dir = 'cropped_images'

# Call the function to perform OCR on the cropped images
ocr_on_cropped_images(cropped_images_dir)


Class: Address
Extracted Text: 
--------------------------------------------------
Class: Class
Extracted Text: 
--------------------------------------------------
Class: DOB
Extracted Text: 
--------------------------------------------------
Class: Exp date
Extracted Text: 
--------------------------------------------------
Class: First name
Extracted Text: 
--------------------------------------------------
Class: Issue date
Extracted Text: 
--------------------------------------------------
Class: Last name
Extracted Text: 
--------------------------------------------------
Class: License number
Extracted Text: 
--------------------------------------------------
Class: Sex
Extracted Text: 
--------------------------------------------------


In [32]:
# !pip install easyocr==1.1.4

In [33]:
# !pip install --upgrade easyocr

In [34]:
# !pip install torch torchvision torchaudio

In [35]:
# !pip install transformers datasets torch torchvision

In [36]:
# !pip install ipywidgets

In [37]:
# !pip install sentencepiece

In [38]:
# !pip install --upgrade transformers sentencepiece

In [55]:
import os
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

extract_classes = ['First name', 'Last name', 'License number', 'Exp date']

data = {}
# Check if the directory exists
if not os.path.isdir(cropped_images_dir):
    print(f"Error: The directory {cropped_images_dir} does not exist.")
else:
    # Load the TrOCR model and processor
    processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-stage1")
    model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

    # Initialize a dictionary to store the extracted text class-wise
    extracted_text_dict = {}

    # Loop over each image file in the directory
    for filename in os.listdir(cropped_images_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(cropped_images_dir, filename)
            
            try:
                # Open the image
                image = Image.open(image_path).convert("RGB")
                
                # Process the image
                pixel_values = processor(images=image, return_tensors="pt").pixel_values

                # Generate text prediction
                generated_ids = model.generate(pixel_values, max_new_tokens=100)
                generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

                # Extract class name from filename (modify as needed)
                class_name = filename.split('_')[0]  # Adjust if needed for different file naming

                # Store the extracted text in the dictionary
                if class_name not in extracted_text_dict:
                    extracted_text_dict[class_name] = []
                extracted_text_dict[class_name].append(generated_text.strip())

                # Optionally, print the result to the console
                print(f"Class: {class_name}")
                print(f"Extracted Text: {generated_text.strip()}")
                print("-" * 50)

            except Exception as e:
                print(f"Error processing {filename}: {e}")
                continue

    # After all images are processed, print the dictionary
    print("\nExtracted Text Dictionary Class-Wise:")
    for class_name, texts in extracted_text_dict.items():
        print(f"Class: {class_name}")
        if class_name in extract_classes:
            data[class_name] = ""
        for text in texts:
            print(f"  Extracted Text: {text}")
            if class_name in extract_classes:
                data[class_name] += text
        print("-" * 50)


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

Class: Address
Extracted Text: MANCHESTER, NET03111
--------------------------------------------------
Class: Class
Extracted Text: CHILD
--------------------------------------------------
Class: DOB
Extracted Text: 05/23/2016
--------------------------------------------------
Class: Exp date
Extracted Text: 05/23/2032
--------------------------------------------------
Class: First name
Extracted Text: VALENTINA
--------------------------------------------------
Class: Issue date
Extracted Text: 06/05/2021
--------------------------------------------------
Class: Last name
Extracted Text: VEGA
--------------------------------------------------
Class: License number
Extracted Text: 603-333-3333
--------------------------------------------------
Class: Sex
Extracted Text: E
--------------------------------------------------

Extracted Text Dictionary Class-Wise:
Class: Address
  Extracted Text: MANCHESTER, NET03111
--------------------------------------------------
Class: Class
  Extract

In [40]:
# !pip install transformers --upgrade

In [41]:
# !pip uninstall transformers sentencepiece -y
# !pip install transformers sentencepiece


In [56]:
data['Name'] = data['First name'] +' ' + data['Last name']
del data['First name']
del data['Last name']

In [57]:
data

{'Exp date': '05/23/2032',
 'License number': '603-333-3333',
 'Name': 'VALENTINA VEGA'}

In [58]:
import shutil
import os

# Check if the directory exists
if os.path.exists(cropped_images_dir) and os.path.isdir(cropped_images_dir):
    try:
        # Remove the directory and all of its contents
        shutil.rmtree(cropped_images_dir)
        print(f"'{cropped_images_dir}' has been removed successfully.")
    except Exception as e:
        print(f"Error occurred while removing the directory: {e}")
else:
    print(f"The directory '{cropped_images_dir}' does not exist.")


'cropped_images' has been removed successfully.
